Instalación de Librerías
Para ejecutar este modelo, necesitas las siguientes bibliotecas. Instálalas con el siguiente comando:

In [1]:
!pip install pandas transformers torch scikit-learn numpy


Fase 1: Preprocesamiento de Datos en Inglés

1.1 Cargar datos y mostrar tamaño del dataset

In [2]:
# Importación de librerías necesarias
import pandas as pd
from transformers import BertTokenizer
from sklearn.preprocessing import LabelEncoder
import torch
from transformers import BertForSequenceClassification
from torch.utils.data import DataLoader, random_split
from transformers import AdamW
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

In [3]:
# Cargar datos desde la ubicación especificada
data = pd.read_csv(r'C:\Users\andfe\Documents\Modelo Sentimientos Despliegue\processed_news_dataset.csv')
print("Datos cargados. Tamaño del dataset:", data.shape)


Datos cargados. Tamaño del dataset: (207433, 16)


Esta sección carga los datos y muestra el tamaño del dataset para verificar que se haya importado correctamente.

1.2 Tokenización y Preparación para BERT

In [4]:
# Eliminar filas donde `processed_text` o `category` es NaN y asegurar que todos los textos sean cadenas
data = data.dropna(subset=['processed_text', 'category'])
data = data[data['processed_text'].apply(lambda x: isinstance(x, str))]

print("Datos después de eliminar valores nulos en 'processed_text' y asegurar que sean cadenas. Tamaño del dataset:", data.shape)


# Reducir el dataset al 10% de su tamaño original
sample_fraction = 0.1  
data = data.sample(frac=sample_fraction, random_state=42)
print("Tamaño del dataset reducido:", data.shape)


# Configuración del tokenizador BERT
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)

# Parámetros de tokenización
max_length = 128
input_ids = []
attention_masks = []

# Tokenización de los textos
for text in data['processed_text']:
    encoded_dict = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    # Añadir los tensores resultantes como elementos individuales en las listas
    input_ids.append(encoded_dict['input_ids'].squeeze(0))  
    attention_masks.append(encoded_dict['attention_mask'].squeeze(0))  

Datos después de eliminar valores nulos en 'processed_text' y asegurar que sean cadenas. Tamaño del dataset: (207431, 16)
Tamaño del dataset reducido: (20743, 16)


Aquí, utilizamos el tokenizador de BERT para transformar los textos en una representación compatible. Se ajustan las secuencias a una longitud máxima y se añade relleno para asegurar uniformidad.

1.3 Codificación de Etiquetas

In [5]:
# Codificación de etiquetas
label_encoder = LabelEncoder()
labels = torch.tensor(label_encoder.fit_transform(data['category']))

print("Codificación de etiquetas completada. Clases encontradas:", label_encoder.classes_)


Codificación de etiquetas completada. Clases encontradas: ['ARTS & CULTURE' 'BUSINESS' 'COMEDY' 'CRIME' 'CULTURE & ARTS'
 'DIVERSITY VOICES' 'DIVORCE' 'EDUCATION' 'ENTERTAINMENT' 'ENVIRONMENT'
 'FOOD & DRINK' 'HOME & LIVING' 'IMPACT' 'MEDIA' 'PARENTS' 'POLITICS'
 'RELIGION' 'SCIENCE' 'SPORTS' 'STYLE & BEAUTY' 'TECH' 'TRAVEL' 'VARIETY'
 'WEDDINGS' 'WELLNESS' 'WOMEN' 'WORLD NEWS']


Convertimos las categorías de noticias en etiquetas numéricas, lo que permite que el modelo las entienda.

1.4 Conversión a Tensores

In [6]:
from torch.utils.data import TensorDataset
import torch

# Verificar que todos los elementos sean tensores
input_ids = [torch.tensor(tensor) if not isinstance(tensor, torch.Tensor) else tensor for tensor in input_ids]
attention_masks = [torch.tensor(tensor) if not isinstance(tensor, torch.Tensor) else tensor for tensor in attention_masks]

# Convertir las listas de tensores en un solo tensor apilado
input_ids = torch.stack(input_ids, dim=0)
attention_masks = torch.stack(attention_masks, dim=0)


# Codificación de etiquetas
label_encoder = LabelEncoder()
labels = torch.tensor(label_encoder.fit_transform(data['category']), dtype=torch.long)


# Verificar que los tamaños coinciden
print("Tamaño de input_ids:", input_ids.size())
print("Tamaño de attention_masks:", attention_masks.size())
print("Tamaño de labels:", labels.size())

# Creación del dataset
dataset = TensorDataset(input_ids, attention_masks, labels)
print("Conversión a tensores completada. Dataset preparado.")

Tamaño de input_ids: torch.Size([20743, 128])
Tamaño de attention_masks: torch.Size([20743, 128])
Tamaño de labels: torch.Size([20743])
Conversión a tensores completada. Dataset preparado.


Agrupamos los datos en un TensorDataset, una estructura que PyTorch requiere para manejar datos en lotes.

Fase 2: Configuración del Modelo BERT

2.1 Carga del Modelo de Clasificación BERT

In [7]:
model = BertForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(label_encoder.classes_),
    output_attentions=False,
    output_hidden_states=False,
)
print("Modelo BERT para clasificación cargado exitosamente.")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Modelo BERT para clasificación cargado exitosamente.


Aquí se carga el modelo BERT preentrenado y se adapta para la clasificación de múltiples categorías, con el número de etiquetas igual a las categorías de noticias.

2.2 Configuración de DataLoaders

In [8]:
batch_size = 16
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

print(f"DataLoaders configurados. Tamaño del conjunto de entrenamiento: {train_size}, tamaño de validación: {val_size}")


DataLoaders configurados. Tamaño del conjunto de entrenamiento: 16594, tamaño de validación: 4149


Aquí dividimos los datos en conjuntos de entrenamiento y validación, y configuramos los DataLoaders para manejar el procesamiento en lotes durante el entrenamiento y validación.

Fase 3: Entrenamiento del Modelo

3.1 Configuración del Optimizador AdamW

In [9]:
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
print("Optimizador AdamW configurado con tasa de aprendizaje:", 2e-5)


C:\Users\andfe\anaconda3\Lib\site-packages\transformers\optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Optimizador AdamW configurado con tasa de aprendizaje: 2e-05


Definimos el optimizador AdamW, que es especialmente adecuado para ajustar modelos BERT, con una tasa de aprendizaje recomendada para mantener la estabilidad del entrenamiento.

3.2 Entrenamiento del Modelo

In [12]:
epochs = 1
max_steps_per_epoch = 20  

# Entrenamiento del modelo
for epoch in range(epochs):
    model.train()
    total_loss = 0
    
    for step, batch in enumerate(train_dataloader):
        if step >= max_steps_per_epoch:
            break
        
        b_input_ids, b_input_mask, b_labels = tuple(t.to(device) for t in batch)
        model.zero_grad()
        
        outputs = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs.loss
        total_loss += loss.item()
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
    
    avg_train_loss = total_loss / (step + 1)  
    print(f"Epoch {epoch+1} - Pérdida Promedio de Entrenamiento: {avg_train_loss:.4f}")
    
    # Validación después de cada época
    model.eval()
    val_accuracy = 0
    for batch in val_dataloader:
        b_input_ids, b_input_mask, b_labels = tuple(t.to(device) for t in batch)
        with torch.no_grad():
            outputs = model(b_input_ids, attention_mask=b_input_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=1).flatten()
        val_accuracy += (predictions == b_labels).cpu().numpy().mean()
    
    avg_val_accuracy = val_accuracy / len(val_dataloader)
    print(f"Precisión Promedio en Validación para Epoch {epoch+1}: {avg_val_accuracy:.4f}")


Epoch 1 - Pérdida Promedio de Entrenamiento: 1.8881
Precisión Promedio en Validación para Epoch 1: 0.5011


El modelo se entrena por varias épocas. En cada época, se calcula y muestra la pérdida de entrenamiento promedio. Después de cada época, se evalúa el modelo en el conjunto de validación, mostrando la precisión promedio.

Fase 4: Evaluación del Modelo

4.1 Evaluación en el Conjunto de Validación

In [13]:
model.eval()
predictions, true_labels = [], []

for batch in val_dataloader:
    b_input_ids, b_input_mask, b_labels = tuple(t.to(device) for t in batch)
    
    with torch.no_grad():
        outputs = model(b_input_ids, attention_mask=b_input_mask)
    
    logits = outputs.logits
    predictions.append(logits.argmax(dim=1).cpu().numpy())
    true_labels.append(b_labels.cpu().numpy())

predictions = np.concatenate(predictions)
true_labels = np.concatenate(true_labels)

print("Reporte de Clasificación:\n", classification_report(true_labels, predictions, target_names=label_encoder.classes_))
print("Matriz de Confusión:\n", confusion_matrix(true_labels, predictions))


Reporte de Clasificación:
                   precision    recall  f1-score   support

  ARTS & CULTURE       0.00      0.00      0.00        32
        BUSINESS       0.54      0.33      0.41       159
          COMEDY       0.00      0.00      0.00        99
           CRIME       0.67      0.03      0.06        66
  CULTURE & ARTS       0.00      0.00      0.00        57
DIVERSITY VOICES       0.42      0.08      0.13       222
         DIVORCE       0.87      0.16      0.27        82
       EDUCATION       0.00      0.00      0.00        44
   ENTERTAINMENT       0.40      0.80      0.53       351
     ENVIRONMENT       0.00      0.00      0.00        69
    FOOD & DRINK       0.54      0.83      0.66       155
   HOME & LIVING       0.69      0.10      0.18       106
          IMPACT       0.00      0.00      0.00        62
           MEDIA       0.00      0.00      0.00        63
         PARENTS       0.57      0.55      0.56       277
        POLITICS       0.60      0.84      0

C:\Users\andfe\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\andfe\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\andfe\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Para la evaluación final, se generan el reporte de clasificación, que incluye métricas como precisión, recall y F1-score, y la matriz de confusión para visualizar el rendimiento por categoría.

